# Importing Necessary Libraries

In [2]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split

# Training

## Loading and Splitting the Data

In [5]:
X = np.load("../Features.npy")
y = np.load("../Labels (Mutli Class).npy")

In [6]:
X.shape, y.shape

((109416, 250, 2), (109416,))

In [7]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, shuffle = True, random_state = 42, test_size = 0.2)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, shuffle = True, random_state = 42, test_size = 0.2)

In [8]:
X_train.shape, X_val.shape, X_test.shape

((70025, 250, 2), (17507, 250, 2), (21884, 250, 2))

In [9]:
# Numberof classes
np.unique(y_train).shape

(14,)

## Model Definition, Compilation and Declaration

In [12]:
def build_ecg_cnn_model(input_shape=(250, 2), num_classes=14):
    model = Sequential([
        Conv1D(filters=32, kernel_size=5, activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        
        Conv1D(filters=64, kernel_size=5, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        
        Conv1D(filters=128, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),

        Conv1D(filters=256, kernel_size=3, activation='relu'),
        BatchNormalization(),
        MaxPooling1D(pool_size=2),
        Dropout(0.2),
        
        Flatten(),

        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.5),
        
        Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.3),
        
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )
    
    return model

In [13]:
model = build_ecg_cnn_model()

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 246, 32)           352       
                                                                 
 batch_normalization (BatchN  (None, 246, 32)          128       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 123, 32)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 123, 32)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 119, 64)           10304     
                                                                 
 batch_normalization_1 (Batc  (None, 119, 64)          2

## Fitting The Model

In [24]:
early_stop = EarlyStopping(monitor = "val_loss", patience = 3, restore_best_weights = True)

In [26]:
history = model.fit(X_train, y_train, validation_data = (X_val, y_val), epochs = 50, batch_size = 64)

Epoch 1/50
1095/1095 [==============================] - 12s 9ms/step - loss: 1.2469 - accuracy: 0.8396 - val_loss: 0.8744 - val_accuracy: 0.9269
Epoch 2/50
1095/1095 [==============================] - 10s 9ms/step - loss: 0.8445 - accuracy: 0.9315 - val_loss: 0.7022 - val_accuracy: 0.9563
Epoch 3/50
1095/1095 [==============================] - 10s 9ms/step - loss: 0.7022 - accuracy: 0.9505 - val_loss: 0.5937 - val_accuracy: 0.9670
Epoch 4/50
1095/1095 [==============================] - 11s 10ms/step - loss: 0.5946 - accuracy: 0.9597 - val_loss: 0.5135 - val_accuracy: 0.9737
Epoch 5/50
1095/1095 [==============================] - 11s 10ms/step - loss: 0.5041 - accuracy: 0.9657 - val_loss: 0.4248 - val_accuracy: 0.9769
Epoch 6/50
1095/1095 [==============================] - 11s 10ms/step - loss: 0.4241 - accuracy: 0.9709 - val_loss: 0.3600 - val_accuracy: 0.9790
Epoch 7/50
1095/1095 [==============================] - 10s 10ms/step - loss: 0.3611 - accuracy: 0.9745 - val_loss: 0.3065 - va

## Saving The Model

In [28]:
# Saving the model in .h5 format
model.save("../Models/Model 1D.h5")

In [29]:
# Saving the model in .keras format
model.save("../Models/Model 1D.keras")

In [30]:
# Saving the model in tf format
model.save("../Model 1D", save_format = "tf")

INFO:tensorflow:Assets written to: ../Model 1D\assets


INFO:tensorflow:Assets written to: ../Model 1D\assets


In [31]:
hist_df = pd.DataFrame(history.history)

In [32]:
hist_df.to_csv("../History 1D.csv")